# evtl. TO DO --> Pfade anpassen

In [ ]:
data_dir=r'/content/drive/MyDrive/ComputerVision/' # Hier sind die Daten gespeichert. Der Ordner "ComputerVision" sollte jeweilis zu jeder Klasse einen unterordner enthalten -->zB. ComptuerVision/Auto und CoputerVison/keinAuto
model_pfad = r"" # in diesem Ordner werden die besten modelle des modeltrainings abgespeichert

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importe
Evtl. müssen hier bibliotheken installiert werden, wie z.B. pip install keras_tuner

In [ ]:
import tensorflow as tf
import time
from time import localtime
import os
import cv2
import imghdr
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from tensorflow.keras.models import load_model
from sklearn.model_selection import StratifiedShuffleSplit
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.metrics import precision_score, accuracy_score, recall_score,confusion_matrix

# Prüfung ob GPU genutzt wird --> siehe Ausgabe der Zelle

In [ ]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

seed_value = 0
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

GPU is available


In [ ]:
BATCH_SIZE = 32
# bei vortrainiertem Model PIC_SIZE nicht ändern!!!
PIC_SIZE = 224 # Annahme viereckiges Bild zb. WEnn PIC_SIZE=500 dann sind die bilder 500x500 pixel;
EPOCH_ANZAHL = 50

# Nicht zu dokumentieren
größe_trainingsdatensatz = 0.2 # prozentuale größe des trainingsdatensatz
größe_validierungsdatensatz = 0.2 # prozentuale größe des validierungsdatensatz

# Validierung des Models

In [ ]:
def validiere_model(model, X_data, y_data):
    prediction = model.predict(X_data)
    acc = accuracy_score(y_data, prediction)
    precision = precision_score(y_data, prediction)
    recall = recall_score(y_data, prediction)
    conf_matrix = confusion_matrix(y_data, prediction)

    print("Accuracy: ",acc, "\n")
    print("Precision: ",precision, "\n")
    print("Recall: ",recall, "\n")

    plt.figure(figsize=(8, 6))
    plt.imshow(conf_matrix, cmap='Blues', interpolation='nearest')
    plt.colorbar()
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# Daten hochladen

In [ ]:
image_exts = ['jpeg', 'jpg', 'png']
# data_dir = 'data' # Für Windows-Rechner

for image_class in os.listdir(data_dir):
    class_path = os.path.join(data_dir, image_class)

    if os.path.isdir(class_path):
        for image in os.listdir(class_path):
            image_path = os.path.join(class_path, image)

            if os.path.isfile(image_path):
                _, file_ext = os.path.splitext(image_path) # check the file ending
                file_ext = file_ext[1:].lower()
                image_type = imghdr.what(image_path) # imghdr checks out if it is a image file

                if file_ext not in image_exts or not image_type:
                    print('File not in image extensions list or not a valid image: {}'.format(image_path))
                    os.remove(image_path)

# Preprocessing data

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(PIC_SIZE, PIC_SIZE),
    label_mode="binary",
    shuffle=True,
    seed=0
)

Found 852 files belonging to 2 classes.


## Daten splitten in Train, Test, val

In [ ]:
# umwandlung in numpy array um die funktion StratifiedShuffleSplit von sklearn auf den datensatz anwenden zu können. Dadurch wird der das verhältnis der klassen bewahrt
features = []
labels = []
for image_batch, label_batch in data:
    features.extend(image_batch.numpy())
    labels.extend(label_batch.numpy())

features = np.array(features)
labels = np.array(labels)

# abspaltung testdaten
abspaltung_test = StratifiedShuffleSplit(n_splits=1, test_size=größe_trainingsdatensatz, random_state=0)
train_val_index, test_index = next(abspaltung_test.split(features, labels))
X_train_val, X_test = features[train_val_index], features[test_index]
y_train_val, y_test = labels[train_val_index], labels[test_index]

# aufteilung val und train
aufteilung_train_val = StratifiedShuffleSplit(n_splits=1, test_size=größe_validierungsdatensatz, random_state=0)
train_index, val_index = next(aufteilung_train_val.split(X_train_val, y_train_val))

# Aufteilung der Trainings- und Validierungsdaten
X_train, X_val = X_train_val[train_index], X_train_val[val_index]
y_train, y_val = y_train_val[train_index], y_train_val[val_index]

X_train = preprocess_input(X_train)
X_val= preprocess_input(X_val)
X_test = preprocess_input(X_test)

X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape


((544, 224, 224, 3),
 (544, 1),
 (137, 224, 224, 3),
 (137, 1),
 (171, 224, 224, 3),
 (171, 1))

# Model Hochladen und validieren

In [ ]:
model = load_model(model_pfad)

In [ ]:
validiere_model(model, X_train, y_train)

In [ ]:
validiere_model(model, X_val, y_val)

In [ ]:
validiere_model(model, X_test, y_test)

# Plotte falsch klassifizierte Bilder

In [ ]:
def plot_falsch_klassifizierte_bilder(model, X_data, y_data):
    predictions = model.predict(X_data)
    predicted_labels = np.argmax(predictions, axis=1)

    falsch_klassifizierte_index = np.where(predicted_labels != y_data)[0]

    for img_index in falsch_klassifizierte_index:
        plt.figure()
        plt.imshow(X_data[img_index])
        plt.title(f"True: {y_data[img_index]}, Predicted: {y_data[img_index]}")
        plt.axis('off')
        plt.show()

In [ ]:
plot_falsch_klassifizierte_bilder(model, X_train, y_train)

In [ ]:
plot_falsch_klassifizierte_bilder(model, X_test, y_test)

In [ ]:
plot_falsch_klassifizierte_bilder(model, X_val, y_val)